# Time series differentiation

## Introduction

Time series differentiation involves computing the differences between consecutive observations in the time series. When training forecasting models, differentiation allows to focus on the relative change ratios instead of directly modeling the absolute values. Once the predictions are estimated, the transformation can be reversed to obtain the values in the original scale.

This is very useful when dealing with time series that shows a trend and Tree-based models. Tree-based models, including decision trees, random forests and gradient boosting machines (GBMs), are known for their effectiveness and widespread use in various machine learning applications. However, they have limitations when it comes to extrapolation, i.e. making predictions or estimates beyond the range of observed data. Because these models cannot predict values beyond the observed range during training, their predicted values will deviate from the underlying trend.

The **skforecast** library, version 0.10.0 or higher, introduces a novel `differentiation` parameter within its forecaster classes to indicate that a differentiation process must be applied before training the model. This is achieved by making internal use of a new transformer named `skforecast.preprocessing.TimeSeriesDifferentiator`. It is worth noting that the entire differentiation process has been automated and its effects are seamlessly reversed during the prediction phase. This ensures that the resulting forecast values are in the original scale of the time series data.

This document shows how differentiation allows to model time series with a positive trend using tree-based models (random forest and a gradient boosting *xgboost*).